# Project

Team Name:     XiaoJiLingGui

Kaggle Login:  siqiguo

Student Name:  Siqi Guo

Student ID:    743053

Kaggle Login: 

Student Name:

Student ID: 

Python version used: 2.7

In [15]:
import nltk
from math import log
from collections import defaultdict, Counter
import requests, tarfile
import json

stopwords = set(nltk.corpus.stopwords.words('english')) # wrap in a set() (see below)
stemmer = nltk.stem.PorterStemmer() 
lemmatizer = nltk.stem.wordnet.WordNetLemmatizer() 
sent_segmenter = nltk.data.load('tokenizers/punkt/english.pickle')


In [12]:
def lemmatize_word(word):
    word = word.lower()
    lemma_n = lemmatizer.lemmatize(word,'n')
    lemma_v = lemmatizer.lemmatize(word,'v')
    if len(lemma_n) <= len(lemma_v):
        return lemma_n
    else:
        return lemma_v

In [11]:
def para_accuracy(fname):
    counter = float(0)
    with open(fname) as json_data:
        infile = json.load(json_data)
        length = len(infile)
        for dic in infile:
            question = dic['question']
            ans = dic['text']
            para = dic['answer_paragraph']
            docid = dic['docid']
            
            result = get_result(docid, query)
            if result == para:
                counter += 1
    return counter/length
develfname = 'project_files/devel.json'
acc = para_accuracy(develfname)
    

NameError: global name 'query' is not defined

Select the most relevant paragraph

In [62]:
def select_document(fname, docid):
    with open(fname) as json_data:
        infile = json.load(json_data)
        for doc in infile:
            if doc['docid'] == docid:
                text = doc['text']
    return text

# def extract_query_terms(query, q_type):
#     q_terms = extract_terms(query)
#     # Since HOW queries have many answer types, furthrer analysis required
#     if q_type == 'HOW':
        

def parse_paragraphs(text):
    identifier = 0
    for para in text:
        yield (identifier, para)
        identifier += 1

# def extract_terms(para):
#     terms = set()
#     for token in nltk.word_tokenize(para):
#         if token not in stopwords: # 'in' and 'not in' opera]tions are much faster over sets that lists
#             terms.add(lemmatize_word(token))
#     return terms

def extract_term_freqs(para):
    tfs = Counter()
    for token in nltk.word_tokenize(para):
        if token not in stopwords: # 'in' and 'not in' operations are much faster over sets that lists
            tfs[lemmatize_word(token)] += 1
    return tfs

def compute_para_freqs(para_term_freqs):
    pfs = Counter()
    for tfs in para_term_freqs.values():
        for term in tfs.keys():
            pfs[term] += 1
    return pfs

def get_index(para_term_freqs, para_freqs, M):
    vsm_inverted_index = defaultdict(list)
    for pid, term_freqs in para_term_freqs.items():
        N = sum(term_freqs.values())
        length = 0

        # find tf*idf values and accumulate sum of squares 
        tfidf_values = []
        for term, count in term_freqs.items():
            tfidf = float(count) / N * log(M / float(para_freqs[term]))
            tfidf_values.append((term, tfidf))
            length += tfidf ** 2

        # normalise documents by length and insert into index
        length = length ** 0.5
        for term, tfidf in tfidf_values:
            # note the inversion of the indexing, to be term -> (doc_id, score)
            vsm_inverted_index[term].append([pid, tfidf / length])

    # ensure posting lists are in sorted order (less important here cf above)
    for term, pids in vsm_inverted_index.items():
        pids.sort()
    return vsm_inverted_index
    
def query_vsm(query, index, k=2):
    accumulator = Counter()
    for term in query:
        postings = index[term]
        for pid, weight in postings:
            accumulator[pid] += weight
    return accumulator.most_common(k)

def get_freq_dict(raw_paras):
#     paras = {}
#     for pid, p in raw_paras:
#         terms = extract_terms(p)
#         paras[pid] = terms

    para_term_freqs = {}
    for pid, para in raw_paras:
        term_freqs = extract_term_freqs(para)
        para_term_freqs[pid] = term_freqs
    M = len(para_term_freqs)
    return para_term_freqs, M


# def get_result(docid, query):
#     docfname = 'project_files/documents.json'
#     text = select_document(docfname, docid)
#     raw_paras = list(parse_paragraphs(text))

#     paras, para_term_freqs, M = get_freq_dict(raw_paras)
#     para_freqs = compute_para_freqs(para_term_freqs)
#     results = query_vsm([stemmer.stem(term.lower()) for term in "constant element particle".split()], 
#                         get_index(para_term_freqs, para_freqs, M))
#     return results

In [63]:
def get_result(docid, query):
    docfname = 'project_files/documents.json'
    text = select_document(docfname, docid)
    raw_paras = list(parse_paragraphs(text))

    para_term_freqs, M = get_freq_dict(raw_paras)
    para_freqs = compute_para_freqs(para_term_freqs)
    results = query_vsm([stemmer.stem(term.lower()) for term in "constant element particle".split()], 
                        get_index(para_term_freqs, para_freqs, M))
    return results, raw_paras

In [64]:
result, raw_paras = get_result(0, 'particle, element, constant')
# print raw_paras
print result
print(raw_paras[result[0][0]])

[(0, 0.1541752855626096), (20, 0.08236155155447321)]
(0, u'First recognized in 1900 by Max Planck, it was originally the proportionality constant between the minimal increment of energy, E, of a hypothetical electrically charged oscillator in a cavity that contained black body radiation, and the frequency, f, of its associated electromagnetic wave. In 1905 the value E, the minimal energy increment of a hypothetical oscillator, was theoretically associated by Einstein with a "quantum" or minimal element of the energy of the electromagnetic wave itself. The light quantum behaved in some respects as an electrically neutral particle, as opposed to an electromagnetic wave. It was eventually called the photon.')
